In [1]:
!pip install /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
!pip install /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl

Processing /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
autograd is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=d809b033b23f19c0d43e1260d216a7318a974caed9aa540ddcf00824cfd9a8e3
  Stored in directory: /root/.cache/pip/wheels/6b/b5/e0/4c79e15c0b5f2c15ecf613c720bb20daab20a666eb67135155
Successfully built autograd-gamma
Processing /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl


In [2]:
import numpy as np
import pandas as pd
import polars as pl
from lifelines import KaplanMeierFitter
import xgboost as xgb
import pickle
import joblib

In [3]:
class Pipeline:
    @staticmethod
    def merge_fold(df, fold_path):
        fold_df = pl.read_csv(fold_path)
        df = df.join(fold_df, on="ID", how="inner")
        return df
    
    def create_kaplanmeier_label(df):
        kmf = KaplanMeierFitter()
        kmf.fit(df["efs_time"], df["efs"])
        y = kmf.survival_function_at_times(df["efs_time"]).values
        df = df.with_columns(
            pl.Series("km_label", y)
        )
        return df

In [4]:
df_test = pl.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv")

In [5]:
def c_index_metric(
        y_true: np.ndarray, 
        y_pred: np.ndarray
    ):
    """
    WARNING: y_true will not be used in this function.
    """
    global df_val
    # submission_df = pd.DataFrame(y_pred[:, 0], columns=["prediction"])
    submission_df = pd.DataFrame(y_pred, columns=["prediction"])
    return -score(df_val, submission_df)

In [6]:
xgb_models = []
cat_encoders = []
for i in range(6):
    print(f"Loading model XGB - {i}")
    # model = xgb.XGBRegressor()
    # model.load_model(fname=f"/kaggle/input/cibmtr-xgb-kplm/xgb_kaplanmeier_encode_fold{i}.json")
    model = joblib.load(f"/kaggle/input/cibmtr-xgb-nelson-cv4/xgb_nelson_cv4_fold{i}.joblib")

    xgb_models.append(model)
    cat_encoders.append(joblib.load(f"/kaggle/input/cibmtr-xgb-nelson-cv4/cat_encoder_fold{i}"))

Loading model XGB - 0
Loading model XGB - 1
Loading model XGB - 2
Loading model XGB - 3
Loading model XGB - 4
Loading model XGB - 5


In [7]:
xgb_preds = []
for i in range(6):
    print(f"XGB - Fold {i}")
    model = xgb_models[i]
    cat_encoder = cat_encoders[i]
    
    X_xgb = df_test.select(model.feature_names_in_).to_pandas()
    
    for col in [f for f in X_xgb.select_dtypes(["object"]).columns if f in model.feature_names_in_]:
        X_xgb[col] = X_xgb[col].astype('category')

    X_xgb = cat_encoder.transform(X_xgb)
    pred = model.predict(X_xgb)
    xgb_preds.append(pred)

XGB - Fold 0
XGB - Fold 1
XGB - Fold 2
XGB - Fold 3
XGB - Fold 4
XGB - Fold 5


In [8]:
df_test = df_test.with_columns(
    prediction=pl.Series(np.mean(xgb_preds, axis=0)),
)

In [9]:
df_test.select("ID", "prediction").to_pandas().to_csv("submission.csv", index=False)

In [10]:
pd.read_csv("submission.csv")

,ID,prediction
0,28800,-0.732693
1,28801,-0.466637
2,28802,-0.786942
